In [1]:
import itertools
import nibabel as nb
import numpy as np
import pandas as pd
from tqdm import tqdm
from atlasreader import atlasreader as ar
from nilearn._utils import check_niimg

The Python package you are importing, AtlasReader, is licensed under the
BSD-3 license; however, the atlases it uses are separately licensed under more
restrictive frameworks.
By using AtlasReader, you agree to abide by the license terms of the
individual atlases. Information on these terms can be found online at:
https://github.com/miykael/atlasreader/tree/master/atlasreader/data



Relabel voxels in Yeo's canonical functional network maps to AAL atlas
---
## 1.1 First load voxel data from Yeo:

In [18]:
# For Yeo 17 Networks:
# Read Yeo labels from file, fixing spacing and new/empty lines
Yeo_labels = open('../data/atlases/Yeo/Yeo2011_7Networks_MNI152.txt', 'r')
Yeo_labels = Yeo_labels.read().split('\n')
#Yeo_labels = [Yeo_labels[i][17:] for i in range(len(Yeo_labels)-1)]
Yeo_labels.insert(0,"None") #I believe the areas were numbered 1-7 with "0" reserved for white matter

#Read Yeo matrix
Yeo_template = "../data/atlases/Yeo/Yeo2011_7Networks_MNI152.nii"
Yeo = nb.load(Yeo_template)
Yeo_data = Yeo.get_fdata()

### 1.2 Loop through MNI-152 template:

In [19]:
AAL_data = ar.get_atlas('aal')['image'].get_fdata()

AAL_atlas = ar.get_atlas('aal')

We would like to loop over the pixels and compare them between atlases but, unfortunately, the matrices are cropped to remove large regions full of zeros so the matrices are not 256x256x256 anymore.

We can try doing the same operation in real, xyz space instead of in MNI-type indices. We do so by using `coord_ijk_to_xyz` function in `atlasreader`.

In [20]:
Yeo_affine = check_niimg(Yeo_template).affine
AAL_affine = check_niimg(ar.get_atlas('aal')['image']).affine

### 1.3 Compute the AAL voxel-based distribution of networks per region (very slow)

Try using multiprocessing and see if it speeds up the process:

In [21]:
for label in Yeo_labels:
    print(label)

None
  1     Visual
  2     Somatomotor
  3     Dorsal_Attention
  4     Ventral_Attention
  5     Limbic
  6     Frontoparietal
  7     Default



In [ ]:
from multiprocessing import Pool

nprocesses = 4 # number of parallel workers

# pre-allocate the dictionaries, we will append to these as we loop through the data:
AAL_dict_voxels = {}
AAL_dict_counts = {}

with Pool(processes=nprocesses) as pool:
    for n in tqdm(ar.get_atlas('aal')['labels']['name'], desc = 'pre-allocation'):
        # this creates a dict for each region name in AAL atlas
        # each region's dict will contain network labels or voxelcounts
        # Yeo's networks are numbered, so we append the integer number labels to each network name
        AAL_dict_counts.update({n: {Yeo_label:0 for Yeo_label in Yeo_labels}})
        # this is the voxels for each network
        AAL_dict_voxels.update({n: {Yeo_label:[] for Yeo_label in Yeo_labels}})

    for aal_i in tqdm(range(0, AAL_data.shape[0]), desc = 'Outter Loop'):
        for aal_j in range(0, AAL_data.shape[1]):
            for aal_k in range(0, AAL_data.shape[2]):
                # Get each iterated voxel's labels:
                AAL_voxel_label = AAL_data[aal_i, aal_j, aal_k]
                AAL_voxel_region = ar.get_label(AAL_atlas, AAL_voxel_label)
                
                if AAL_voxel_region != 'no_label':
                    # Excluding the unknown voxels, we re-do the affine conversion
                    # then assign the Yeo labels/voxels to our AAL dict
                    xyz = ar.coord_ijk_to_xyz(AAL_affine, [aal_i, aal_j, aal_k])
                    Yeo_ijk = ar.coord_xyz_to_ijk(Yeo_affine, xyz)[0]
                    
                    Yeo_voxel_label = int(Yeo_data[Yeo_ijk[0], Yeo_ijk[1], Yeo_ijk[2]])
                    Yeo_voxel_region = Yeo_labels[Yeo_voxel_label]
                    AAL_dict_counts[AAL_voxel_region][Yeo_voxel_region] += 1 # add one to voxel count
                    AAL_dict_voxels[AAL_voxel_region][Yeo_voxel_region].append(xyz[0]) # label region


Outter Loop:  29%|██▉       | 22/75 [02:36<06:22,  7.23s/it]

Now that's finished, we want to save it so we can just load it later:

In [ ]:
from scipy.io import savemat

np.save('')